In [1]:
import pandas as pd
import numpy as np
import requests
import string
import collections
import random
import time
import pickle
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textaugment import Wordnet
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_OPTIONAL_SKILLS.csv') 
df_occ_n_skills.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35824 entries, 0 to 35823
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0.1             35824 non-null  int64 
 1   Unnamed: 0               35824 non-null  int64 
 2   conceptType              35824 non-null  object
 3   conceptUri               35824 non-null  object
 4   iscoGroup                35824 non-null  int64 
 5   preferredLabel           35824 non-null  object
 6   altLabels                35823 non-null  object
 7   hiddenLabels             53 non-null     object
 8   status                   35824 non-null  object
 9   modifiedDate             35824 non-null  object
 10  regulatedProfessionNote  35824 non-null  object
 11  scopeNote                3612 non-null   object
 12  definition               89 non-null     object
 13  inScheme                 35824 non-null  object
 14  description              35824 non-nul

/tmp/ipykernel_20470/402935332.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_OPTIONAL_SKILLS.csv')


In [3]:
df_occ_n_skills.head()

,Unnamed: 0.1,Unnamed: 0,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code,skills,opt_skills
0,0,0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2654,technical director,technical and operations director\nhead of tec...,NaN,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,2654.1.7,"adapt to artists' creative demands, promote he...","ESCO Occupations, ESCO member occupations"
1,1,1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,metal drawing machine operator,metal drawing machine operator\nmetal drawing ...,NaN,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Metal drawing machine operators set up and ope...,8121.4,"set up the controller of a machine, monitor mo...","safely handle metal wire under tension, consul..."
2,2,2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,precision device inspector,inspector of precision instruments\nprecision ...,NaN,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Precision device inspectors make sure precisio...,7543.10.3,"troubleshoot, communicate test results to othe...","microprocessors, circuit diagrams, mechanical ..."
3,3,3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,air traffic safety technician,air traffic safety electronics hardware specia...,NaN,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Air traffic safety technicians provide technic...,3155.1,"aircraft flight control systems, electronics, ...","electrical engineering, perform aircraft maint..."
4,4,4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,hospitality revenue manager,hospitality revenues manager\nyield manager\nh...,NaN,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Hospitality revenue managers maximise revenue ...,2431.9,"think analytically, monitor financial accounts...","quote prices, maintain customer service, coach..."


In [4]:
# get dataframes from CSV files

df_occupations = pd.read_csv('../data/ESCO/occupations_en.csv')
df_occupations_aug = pd.read_csv('../data/ESCO/occupations_augmented_with_OPTIONAL_SKILLS.csv')
df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')

/tmp/ipykernel_20470/3666444066.py:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_occupations_aug = pd.read_csv('../data/ESCO/occupations_augmented_with_OPTIONAL_SKILLS.csv')
/tmp/ipykernel_20470/3666444066.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')


In [5]:
# filter unneeded columns of of the dataframe and add needed ones

df_occ_n_skills = df_occ_n_skills.filter(items=['preferredLabel', 'description', 'skills', 'opt_skills'])
df_occ_n_skills = df_occ_n_skills.reindex(columns=['preferredLabel','description', 'skills', 'opt_skills'])
df_occ_n_skills.rename(columns={'preferredLabel': 'job_title'}, inplace=True)
df_occ_n_skills['description_input'] = 0
df_occ_n_skills['skills_input'] = 0

In [6]:
df_occ_n_skills.head()

,job_title,description,skills,opt_skills,description_input,skills_input
0,technical director,Technical directors realise the artistic visio...,"adapt to artists' creative demands, organise r...",NaN,0,0
1,metal drawing machine operator,Metal drawing machine operators set up and ope...,"cold drawing processes, monitor moving workpie...",NaN,0,0
2,precision device inspector,Precision device inspectors make sure precisio...,"precision measuring instruments, monitor machi...",NaN,0,0
3,air traffic safety technician,Air traffic safety technicians provide technic...,"air transport law, aircraft flight control sys...",NaN,0,0
4,hospitality revenue manager,Hospitality revenue managers maximise revenue ...,"develop revenue generation strategies, ensure ...",NaN,0,0


In [7]:
# create description_input and skills_input, which are strings on which the model will be fit

for row, index in tqdm(df_occ_n_skills.iterrows()):
    underscored_job_title = index['job_title'].replace(" ", "_")
    this_rows_description_input = underscored_job_title + ' ' + index['description']
    if type(index['opt_skills']) != str:
        index['opt_skills'] = ''
    this_rows_skills_input = underscored_job_title + ' ' + index['skills'] + ', ' + index['opt_skills']
    df_occ_n_skills.iloc[row,-2] = this_rows_description_input
    df_occ_n_skills.iloc[row,-1] = this_rows_skills_input

35824it [00:15, 2300.07it/s]


In [8]:
df_occ_n_skills

,job_title,description,skills,opt_skills,description_input,skills_input
0,technical director,Technical directors realise the artistic visio...,"adapt to artists' creative demands, organise r...",NaN,technical_director Technical directors realise...,technical_director adapt to artists' creative ...
1,metal drawing machine operator,Metal drawing machine operators set up and ope...,"cold drawing processes, monitor moving workpie...",NaN,metal_drawing_machine_operator Metal drawing m...,metal_drawing_machine_operator cold drawing pr...
2,precision device inspector,Precision device inspectors make sure precisio...,"precision measuring instruments, monitor machi...",NaN,precision_device_inspector Precision device in...,precision_device_inspector precision measuring...
3,air traffic safety technician,Air traffic safety technicians provide technic...,"air transport law, aircraft flight control sys...",NaN,air_traffic_safety_technician Air traffic safe...,air_traffic_safety_technician air transport la...
4,hospitality revenue manager,Hospitality revenue managers maximise revenue ...,"develop revenue generation strategies, ensure ...",NaN,hospitality_revenue_manager Hospitality revenu...,hospitality_revenue_manager develop revenue ge...
...,...,...,...,...,...,...
35819,motor vehicle assemblies tester,motor fomite assemblers install and put prefab...,"work in assembly line teams, vehicle manufactu...",NaN,motor_vehicle_assemblies_tester motor fomite a...,motor_vehicle_assemblies_tester work in assemb...
35820,car factory worker,motor fomite assemblers install and put prefab...,"motor vehicle parts drawings, drive motor vehi...",NaN,car_factory_worker motor fomite assemblers ins...,car_factory_worker motor vehicle parts drawing...
35821,motor vehicle assembly inspector,motor fomite assembler install and put prefabr...,"quality standards, mechanics of motor vehicles...",NaN,motor_vehicle_assembly__inspector motor fomite...,motor_vehicle_assembly__inspector quality stan...
35822,truck production line worker,motor fomite assembler install and put prefabr...,"quality standards, monitor manufacturing quali...",NaN,truck_production_line_worker motor fomite asse...,truck_production_line_worker quality standards...


In [9]:
X_all = pd.concat([df_occ_n_skills['description_input'], df_occ_n_skills['skills_input']]).reset_index(drop=True)

In [10]:
X_all

0        technical_director Technical directors realise...
1        metal_drawing_machine_operator Metal drawing m...
2        precision_device_inspector Precision device in...
3        air_traffic_safety_technician Air traffic safe...
4        hospitality_revenue_manager Hospitality revenu...
                               ...                        
71643    motor_vehicle_assemblies_tester work in assemb...
71644    car_factory_worker motor vehicle parts drawing...
71645    motor_vehicle_assembly__inspector quality stan...
71646    truck_production_line_worker quality standards...
71647    motor_vehicle_body_builder mechanics of motor ...
Length: 71648, dtype: object

In [11]:
# applying preprocessing as in Gensim tutorial, also applying Phraser

def read_corpus(corpus):
   
    # instantiate Phraser outside of the loop
    sentence_stream = [entry.split(" ") for entry in corpus]
    bigrams = Phrases(
        sentence_stream,
        min_count=5,
        threshold=5,
        connector_words=ENGLISH_CONNECTOR_WORDS
        )
    
    for i, line in enumerate(corpus):    
    
        # remove punctuation
        for punctuation in string.punctuation:        
            sentence = line.replace(punctuation, '')

        # remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(sentence)
        stopword_free_tokens = [token for token in tokens if token not in stop_words]
        sentence = ' '.join(stopword_free_tokens)

        # lemmatize
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
        
        # get bigrams
        sent = sentence.split()

        # yield tagged final corpus
        yield TaggedDocument(bigrams[sent], [i])

all_corpus = list(read_corpus(X_all))

In [12]:
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [13]:
CUDA_LAUNCH_BLOCKING=1
all_corpus_embed = bert_model.encode(all_corpus)
CUDA_LAUNCH_BLOCKING=1

KeyboardInterrupt: 

In [ ]:
new_description_embed = bert_model.encode(new_description)

In [ ]:
# save the model to disk
filename = 'bert_model_22.sav'
pickle.dump(bert_model, open(filename, 'wb'))

In [ ]:
# save all_corpus_embedding to disk
filename = 'all_corpus_embed_22.sav'
pickle.dump(all_corpus_embed, open(filename, 'wb'))